In [5]:
import torch
import transformers
from core.data_utils import SCget_data, SCloadCorpus
from core.models import model_selector
from core.data_utils.build_graph import get_data
from sklearn.metrics import accuracy_score, f1_score

from torch.nn import functional as F
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


_basepath = "./data/sentiment/"
_model = "bert"
_bert = "roberta-base"
_max_length = 128
_batch_size = 32
_lr = 1e-4
_epochs=10
_device="cuda:1"


In [6]:
x_tr, y_tr, x_ts, y_ts, x_val, y_val, x_all, y_all, adj = SCget_data(_basepath)
text = SCloadCorpus(_basepath)

y_tr_np = y_tr.toarray()
y_ts_np = y_ts.toarray()

y_tr_orig = y_tr_np[:int(len(y_tr_np)/2), :]
y_tr_new = y_tr_np[int(len(y_tr_np)/2):, :]
y_ts_orig = y_ts_np[:int(len(y_ts_np)/2), :]
y_ts_new = y_ts_np[int(len(y_ts_np)/2):, :]

train_text = text[:x_tr.shape[0]]
val_text = text[x_tr.shape[0]:x_tr.shape[0] + x_val.shape[0]]
test_text = text[x_tr.shape[0] + x_val.shape[0]:]

train_text_orig = train_text[:int(len(y_tr_np)/2)]
train_text_new  = train_text[int(len(y_tr_np)/2):]
test_text_orig  = test_text[:int(len(y_ts_np)/2)]
test_text_new   = test_text[int(len(y_ts_np)/2):]

print(len(train_text_orig), len(train_text_new), len(test_text_orig), len(test_text_new))

train_loader = DataLoader((train_text_orig, y_tr_orig), batch_size=32, shuffle=True)
test_loader = DataLoader((test_text_new, y_ts_new), batch_size=32, shuffle=True)

1707 1707 488 488


In [9]:
model = model_selector(_model)(_bert, 2, 0.5)
model.to(_device)

optim = torch.optim.Adam(model.classifier.parameters(), lr=_lr)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
_input = model.tokenizer(text, max_length=_max_length,
                                    truncation=True, padding='max_length',
                                    return_tensors='pt')
input_ids, attention_mask = _input.input_ids, _input.attention_mask

train_orig_iid, train_orig_attm = input_ids[:int(len(y_tr_np)/2)], attention_mask[:int(len(y_tr_np)/2)]
train_new_iid, train_new_attm = input_ids[int(len(y_tr_np)/2):], attention_mask[int(len(y_tr_np)/2):]
test_orig_iid, test_orig_attm = input_ids[y_tr.shape[0]+y_val.shape[0]:int(len(y_ts_np)/2)], attention_mask[y_tr.shape[0]+y_val.shape[0]:int(len(y_ts_np)/2)]
test_new_iid, test_new_attm = input_ids[y_tr.shape[0]+y_val.shape[0]+int(len(y_ts_np)/2):], attention_mask[y_tr.shape[0]+y_val.shape[0]+int(len(y_ts_np)/2):]

train_data = TensorDataset(train_orig_iid, train_orig_attm, torch.tensor(y_tr_orig))
test_data = TensorDataset(test_new_iid, test_new_attm, torch.tensor(y_ts_new))


train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=True)


In [19]:
for e in range(_epochs):
    train_acc_mean = 0.0
    for i, (ids, attm, y) in enumerate(train_loader):
        model.train()
        ids = ids.to(_device)
        attm = attm.to(_device)
        y = y.to(_device)
        optim.zero_grad()
        pred = model(ids, attm)
        loss = F.nll_loss(pred, torch.argmax(y, axis=1))
        loss.backward()
        optim.step()
        train_loss = loss.item()
        with torch.no_grad():
            model.eval()
            y_true = y.argmax(axis=1).detach().cpu()
            y_pred = pred.argmax(axis=1).detach().cpu()
            train_acc = accuracy_score(y_true, y_pred)
            train_acc_mean += train_acc
        
    train_acc_mean /= len(train_loader)
    
    with torch.no_grad():
        for i, (ids, attm, y) in enumerate(test_loader):
            ids = ids.to(_device)
            attm = attm.to(_device)
            pred = model(ids, attm)
            y_true = y.argmax(axis=1).detach().cpu()
            y_pred = pred.argmax(axis=1).detach().cpu()
            test_acc = accuracy_score(y_pred, y_true)

    print(f"Train acc: {train_acc}, Test acc: {test_acc}")


Train acc: 0.36363636363636365, Test acc: 0.5389344262295082
Train acc: 0.6363636363636364, Test acc: 0.5614754098360656
Train acc: 0.45454545454545453, Test acc: 0.5225409836065574
Train acc: 0.5454545454545454, Test acc: 0.5922131147540983
Train acc: 0.6363636363636364, Test acc: 0.610655737704918
Train acc: 0.7272727272727273, Test acc: 0.6004098360655737
Train acc: 0.5454545454545454, Test acc: 0.6454918032786885
Train acc: 0.45454545454545453, Test acc: 0.5635245901639344
Train acc: 0.8181818181818182, Test acc: 0.6290983606557377
Train acc: 0.6363636363636364, Test acc: 0.6372950819672131
